In [13]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'AppleGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 깨짐 방지

In [14]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


# 1.어뷰징 정의

In [ ]:
import pandas as pd
import numpy as np

list_1 = pd.read_csv('1.광고목록_전처리.csv')
part = pd.read_csv('2.광고참여_전처리.csv')
point = pd.read_csv('3.광고적립_전처리.csv')
time = pd.read_csv('4.시간별_전처리.csv')

# 지표 1번
df_counts = (
    part.groupby(['dvc_idx', 'ads_idx', 'click_date'])
      .size()
      .reset_index(name='cnt')
)

# 2. 동시성 발생 (2회 이상 참여) 필터링
df_abuse = df_counts[(df_counts['cnt'] >= 2) & (df_counts['dvc_idx'] != 0)]


# 3. 고유 디바이스 아이디 리스트
unique_users_1 = df_abuse['dvc_idx'].unique().tolist()

# print("의심 디바이스 수:", len(unique_users_1))
# print("의심 디바이스 예시:", unique_users_1[:20])

# 지표 2번
# 날짜형 변환
point['click_date'] = pd.to_datetime(point['click_date'])
point['regdate'] = pd.to_datetime(point['regdate'])

# 1. 디바이스별 시간 순서대로 정렬
point_2 = point.sort_values(by=['dvc_idx', 'click_date'])

# 2. 디바이스별로 다음 광고 참여 시작 시각 - 이전 광고 완료 시각 계산
point_2['prev_regdate'] = point_2.groupby('dvc_idx')['regdate'].shift(1)
point_2['time_diff'] = (point_2['click_date'] - point_2['prev_regdate']).dt.total_seconds()

# 3. 3초 미만 필터링
df_fast = point_2[point_2['time_diff'] < 3]
df_fast = df_fast[df_fast['dvc_idx'] != 0]

# 4. 고유 디바이스 아이디 리스트
unique_users_2 = df_fast['dvc_idx'].unique().tolist()

# print("의심 디바이스 수:", len(unique_users_2))
# print("의심 디바이스 예시:", unique_users_2[:20])

# 지표 3번
list_3 = list_1[['ads_idx','ads_type']]
point_3 = point[['ads_idx','dvc_idx','ctit','click_date']]
df_3 = pd.merge(point_3, list_3, how='left',on='ads_idx')
df_3 = df_3[df_3['dvc_idx'] != 0]
df_3['click_date'] = pd.to_datetime(df_3['click_date'])
df_3['hour'] = df_3['click_date'].dt.hour
df_3['date'] = df_3['click_date'].dt.date
df_3['ads_type'].value_counts()

df_3_1 = df_3[df_3['ads_type'] == 1]
df_3_5 = df_3[df_3['ads_type'] == 5]
df_3_7 = df_3[df_3['ads_type'] == 7]
df_3_11 = df_3[df_3['ads_type'] == 11]

# 설치형(1) 광고 의심/정상
df_3_ctit_mean = df_3_1.groupby('dvc_idx').agg(ctit_mean=('ctit','mean'))
df_3_1 = pd.merge(df_3_1, df_3_ctit_mean, how='left',on='dvc_idx')
print('고유 아닌 설치형 데이터 갯수: ',df_3_1.shape)
df_3_1 = df_3_1.drop_duplicates(subset='dvc_idx',keep='first')

df_3_1['label'] = df_3_1['ctit_mean'].apply(lambda x: '의심' if x < 10 else '정상')
df_3_1['label'].value_counts()

type1_users_ctit = df_3_1[df_3_1['label'] == '의심']['dvc_idx'].unique().tolist()
# print('너무 짧은 ctit로 의심되는 고유 디바이스 수(설치형):', len(type1_users_ctit))
# print(type1_users_ctit[:20])

# 페북(5) 의심/정상
df_3_ctit_mean = df_3_5.groupby('dvc_idx').agg(ctit_mean=('ctit','mean'))
df_3_5 = pd.merge(df_3_5, df_3_ctit_mean, how='left',on='dvc_idx')
# print('고유 아닌 참여형 데이터 갯수: ',df_3_5.shape)
df_3_5 = df_3_5.drop_duplicates(subset='dvc_idx',keep='first')
df_3_5

# IQR 계산
q1 = np.percentile(df_3_5['ctit_mean'], 25)
q3 = np.percentile(df_3_5['ctit_mean'], 75)
iqr = q3 - q1
bound_기준 = q1 - (1.5 * iqr)
df_3_5['type5_의심'] = (df_3_5['ctit_mean'] < bound_기준)
df_3_5['label'] = '정상'
df_3_5.loc[df_3_5['type5_의심'], 'label'] = '의심'
df_3_5['label'].value_counts()

type5_users_ctit = df_3_5[df_3_5['label'] == '의심']['dvc_idx'].unique().tolist()
# print('너무 짧은 ctit로 의심되는 고유 디바이스 수(페북형):', len(type5_users_ctit))
# print(type5_users_ctit[:20])

# 인스타(7) 의심/정상
df_3_ctit_mean = df_3_7.groupby('dvc_idx').agg(ctit_mean=('ctit','mean'))
df_3_7 = pd.merge(df_3_7, df_3_ctit_mean, how='left',on='dvc_idx')
# print('고유 아닌 인스타 데이터 갯수: ',df_3_7.shape)
df_3_7 = df_3_7.drop_duplicates(subset='dvc_idx',keep='first')
df_3_7

# IQR 계산
q1 = np.percentile(df_3_7['ctit_mean'], 25)
q3 = np.percentile(df_3_7['ctit_mean'], 75)
iqr = q3 - q1
bound_기준 = q1 - (1.5 * iqr)
df_3_7['type7_의심'] = (df_3_7['ctit_mean'] < bound_기준)
df_3_7['label'] = '정상'
df_3_7.loc[df_3_7['type7_의심'], 'label'] = '의심'
df_3_7['label'].value_counts()

type7_users_ctit = df_3_7[df_3_7['label'] == '의심']['dvc_idx'].unique().tolist()
# print('너무 짧은 ctit로 의심되는 고유 디바이스 수(인스타):', len(type7_users_ctit))
# print(type7_users_ctit[:20])

# 네이버(11) 의심/정상
df_3_ctit_mean = df_3_11.groupby('dvc_idx').agg(ctit_mean=('ctit','mean'))
df_3_11 = pd.merge(df_3_11, df_3_ctit_mean, how='left',on='dvc_idx')
# print('고유 아닌 네이버 데이터 갯수: ',df_3_11.shape)
df_3_11 = df_3_11.drop_duplicates(subset='dvc_idx',keep='first')
df_3_11

# IQR 계산
q1 = np.percentile(df_3_11['ctit_mean'], 25)
q3 = np.percentile(df_3_11['ctit_mean'], 75)
iqr = q3 - q1
bound_기준 = q1 - (1.5 * iqr)
df_3_11['type11_의심'] = (df_3_11['ctit_mean'] < bound_기준)
df_3_11['label'] = '정상'
df_3_11.loc[df_3_11['type11_의심'], 'label'] = '의심'
df_3_11['label'].value_counts()

type11_users_ctit = df_3_11[df_3_11['label'] == '의심']['dvc_idx'].unique().tolist()
# print('너무 짧은 ctit로 의심되는 고유 디바이스 수(네이버):', len(type11_users_ctit))
# print(type11_users_ctit[:20])

# 광고유형 1,5,7,11번 유저 합치기(중복X)
all_users_3 = type1_users_ctit + type5_users_ctit + type7_users_ctit + type11_users_ctit
# 중복 제거 (set → 다시 list 변환)
unique_users_3 = list(set(all_users_3)) # 3번 최종 유저 리스트
# print("총 개수:", len(unique_users_3))
# print(unique_users_3[:20]) 

# 지표 4번
list_4 = list_1[['ads_idx','ads_rejoin_type']]
part_4 = part[['ads_idx','dvc_idx','click_date']]
df_4 = pd.merge(part_4, list_4, how='left',on='ads_idx')
df_4 = df_4[df_4['dvc_idx'] != 0]

# ads_rejoin_type 12개 결측치 -> 결측치 삭제
df_4 = df_4.dropna(axis=0)
df_4['click_date'] = pd.to_datetime(df_4['click_date'])

# 1. 재참여 불가 광고인데 동일 dvc_idx에서 반복 참여
# 하루에 3번 이상 참여하면 어뷰징
df_4['date'] = df_4['click_date'].dt.date
df_4_dvc = df_4[df_4['ads_rejoin_type'] != 'REJOINABLE'].groupby(['dvc_idx','ads_idx','date']).size().reset_index(name='count').query('count >= 3').sort_values(by='count',ascending=False).reset_index(drop=True)
df_4_dvc1 = df_4_dvc['dvc_idx'].unique().tolist()
# print('재참여 불가 광고를 하루 3번 이상 반복 참여한 디바이스 수 :',len(df_4_dvc1))
# print(df_4_dvc1[:20])

# 2. 재참여 가능 광고라도 짧은 간격(3초 미만) 반복 참여
df_4_1 = df_4[df_4['ads_rejoin_type'] == 'REJOINABLE'].groupby(['dvc_idx','ads_idx']).size().reset_index(name='count').query('count > 1').sort_values(by='count', ascending=False).reset_index(drop=True)
df_4_1 = pd.merge(df_4_1, df_4, how='left', on=['dvc_idx','ads_idx'])
df_4_1 = df_4_1.sort_values(by=['dvc_idx','ads_idx','click_date'])    
df_4_1['time_diff'] = df_4_1.groupby(['dvc_idx', 'ads_idx'])['click_date'].diff().dt.total_seconds()
df_4_1[df_4_1['time_diff'] <= 2].groupby(['dvc_idx','ads_idx','time_diff']).size().reset_index(name='time_diff_count').sort_values(by='time_diff_count', ascending=False).reset_index(drop=True)

fast_repeat = df_4_1[df_4_1['time_diff'] <= 2]['dvc_idx'].unique()
# print('재참여 가능 광고를 3초 미만 간격으로 반복 참여한 디바이스 수 :', len(fast_repeat))
# print(fast_repeat[:20])

all_users_4 = list(df_4_dvc1) + list(fast_repeat)
# 중복 제거
unique_users_4 = list(set(all_users_4))  # 4번 최종 유저 리스트
# print("총 개수:", len(unique_users_4))
# print(unique_users_4[:20])

# 지표 5번
list_5 = list_1[['ads_idx','ads_type']]
part_5 = part[['ads_idx','dvc_idx','reward_price','contract_price']]
df_5 = pd.merge(part_5, list_5, how='left', on='ads_idx')
df_5 = df_5[df_5['dvc_idx'] != 0]

# 전체 참여 광고 중 타겟 상위 10% 광고를 참여한 상위 10% 유저 추출 
df_5['reward_ratio'] = np.where(df_5['contract_price'] == 0, df_5['reward_price'], df_5['reward_price'] / df_5['contract_price'])
reward_ratio_10 = df_5['reward_ratio'].quantile(0.9)
df_reward_ratio_10 = df_5[df_5['reward_ratio'] >= reward_ratio_10].round(2).reset_index(drop=True)
top10 = df_reward_ratio_10['ads_idx'].unique()

part_cnt = df_reward_ratio_10.groupby('dvc_idx')['ads_idx'].nunique().reset_index(name='count')
part_cnt['part_ratio'] = part_cnt['count'] / top10.size
part_cnt = part_cnt.sort_values(by='part_ratio', ascending=False).reset_index(drop=True)
user_10 = part_cnt['part_ratio'].quantile(0.9)
unique_users_5 = part_cnt[part_cnt['part_ratio'] >= user_10]['dvc_idx'].unique()

# print('전체 참여 광고 중 타겟 상위 10% 광고를 참여한 상위 10% 유저:', len(unique_users_5))
# print(unique_users_5[:20])

lists = {
    "unique_users_1": list(unique_users_1),
    "unique_users_2": list(unique_users_2),
    "unique_users_3": list(unique_users_3),
    "unique_users_4": list(unique_users_4),
    "unique_users_5": list(unique_users_5)
}

all_users = []
for name, lst in lists.items():
    for user in lst:
        all_users.append((user, name)) #user = dvc_idx, lst_name = source 

df_users = pd.DataFrame(all_users, columns=["dvc_idx", "source"])

df_counts = df_users.groupby("dvc_idx").size().reset_index(name="count")
df_counts

In [16]:
# count 컬럼이 3,4,5 -> 어뷰징 리스트 (약 4%)
df_cnt = df_counts[~df_counts['count'].isin([1, 2])]


# 2. EDA

In [ ]:
# -----어뷰징 평균 리워드 > 정상 평균 리워드 ---------


import pandas as pd
import matplotlib.pyplot as plt

# 1️⃣ 어뷰징 디바이스 리스트
abuse_dvc = df_cnt['dvc_idx'].unique()

# 2️⃣ point + list_1 merge (ads_type 붙이기)
point_merged = point.merge(
    list_1[['ads_idx', 'ads_type']], 
    on='ads_idx', 
    how='left'
)

# 3️⃣ 시간대(hour) 컬럼 생성
point_merged['regdate'] = pd.to_datetime(point_merged['regdate'])
point_merged['hour'] = point_merged['regdate'].dt.hour

# ✅ 4️⃣ combo를 광고유형-매체사까지만 생성
point_merged['combo'] = (
    point_merged['ads_type'].astype(str) + "-" +
    point_merged['mda_idx'].astype(str)
)

# 5️⃣ 그룹 라벨 (어뷰징 / 정상)
point_merged['group'] = point_merged['dvc_idx'].isin(abuse_dvc).map({True:'abuse', False:'normal'})

# 6️⃣ combo × group별 평균 리워드 계산
combo_group_stats = (
    point_merged.groupby(['combo','group'])['rwd_cost']
    .mean()
    .reset_index()
)

# 7️⃣ pivot으로 abuse/normal 나란히 배치
combo_pivot = combo_group_stats.pivot(index='combo', columns='group', values='rwd_cost').fillna(0)
combo_pivot = combo_pivot.reset_index().rename(columns={'abuse':'avg_rwd_abuse','normal':'avg_rwd_normal'})

# 8️⃣ 어뷰징 평균 리워드 > 정상 평균 리워드 필터링 + diff 계산
higher_abuse = combo_pivot[combo_pivot['avg_rwd_abuse'] > combo_pivot['avg_rwd_normal']]
higher_abuse['diff'] = higher_abuse['avg_rwd_abuse'] - higher_abuse['avg_rwd_normal']

# 9️⃣ diff 큰 순으로 Top10
top10_combo = higher_abuse.sort_values('diff', ascending=False).head(10)

print(top10_combo[['combo','avg_rwd_abuse','avg_rwd_normal','diff']])

# 🔟 시각화
plt.figure(figsize=(14,5))
bar_width = 0.4
x = range(len(top10_combo))

plt.bar([i - bar_width/2 for i in x], top10_combo['avg_rwd_abuse'], 
        width=bar_width, color='red', label='어뷰징 평균 리워드')
plt.bar([i + bar_width/2 for i in x], top10_combo['avg_rwd_normal'], 
        width=bar_width, color='blue', label='정상 평균 리워드')

plt.xticks(x, top10_combo['combo'], rotation=45, ha='right')
plt.title("어뷰징 평균 리워드 > 정상 평균 리워드 (광고유형-매체사 기준) Top10")
plt.xlabel("광고유형-매체사")
plt.ylabel("평균 rwd_cost")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# -----어뷰징 디바이스 수 > 정상 디바이스 수 (적립) ---------

import pandas as pd
import matplotlib.pyplot as plt

# 1️⃣ 어뷰징 디바이스 리스트
abuse_dvc = df_cnt['dvc_idx'].unique()

# 2️⃣ point + list_1 merge
point_merged = point.merge(
    list_1[['ads_idx','ads_type']],
    on='ads_idx',
    how='left'
)

# 3️⃣ 시간대 & combo 생성
point_merged['regdate'] = pd.to_datetime(point_merged['regdate'])
point_merged['hour'] = point_merged['regdate'].dt.hour

point_merged['combo'] = (
    point_merged['ads_type'].astype(str) + "-" +
    point_merged['mda_idx'].astype(str) 
)

# 4️⃣ 그룹 라벨
point_merged['group'] = point_merged['dvc_idx'].isin(abuse_dvc).map({True:'abuse', False:'normal'})

# 5️⃣ combo × group별 고유 디바이스 수 집계
combo_device_stats = (
    point_merged.groupby(['combo','group'])['dvc_idx']
    .nunique()
    .reset_index()
)

# 6️⃣ pivot으로 abuse/normal 나란히
combo_pivot = combo_device_stats.pivot(index='combo', columns='group', values='dvc_idx').fillna(0)
combo_pivot = combo_pivot.reset_index().rename(columns={'abuse':'abuse_devices','normal':'normal_devices'})

# 7️⃣ 어뷰징 디바이스 수 > 정상 디바이스 수 필터링
combo_pivot = combo_pivot[combo_pivot['abuse_devices'] > combo_pivot['normal_devices']]

# 8️⃣ 차이(diff) 계산 & Top 10
combo_pivot['diff'] = combo_pivot['abuse_devices'] - combo_pivot['normal_devices']
top10_combo = combo_pivot.sort_values('diff', ascending=False).head(10)

print(top10_combo[['combo','abuse_devices','normal_devices','diff']])


plt.figure(figsize=(14,5))
bar_width = 0.4
x = range(len(top10_combo))

plt.bar([i - bar_width/2 for i in x], top10_combo['abuse_devices'], width=bar_width, color='red', label='어뷰징')
plt.bar([i + bar_width/2 for i in x], top10_combo['normal_devices'], width=bar_width, color='blue', label='정상')

plt.xticks(x, top10_combo['combo'], rotation=45, ha='right')
plt.title("어뷰징 디바이스 수 > 정상 디바이스 수 콤보 Top10")
plt.xlabel("광고유형-매체사-시간대")
plt.ylabel("고유 디바이스 수")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# -----어뷰징 리워드 합계 > 정상 리워드 합계 ---------

import pandas as pd
import matplotlib.pyplot as plt

# 1️⃣ 어뷰징 디바이스 리스트
abuse_dvc = df_cnt['dvc_idx'].unique()

# 2️⃣ point + list_1 merge
point_merged = point.merge(
    list_1[['ads_idx','ads_type']], 
    on='ads_idx', 
    how='left'
)

# 3️⃣ combo(광고유형-매체사) 생성
point_merged['combo'] = point_merged['ads_type'].astype(str) + "-" + point_merged['mda_idx'].astype(str)

# 4️⃣ 그룹 라벨
point_merged['group'] = point_merged['dvc_idx'].isin(abuse_dvc).map({True:'abuse', False:'normal'})

# 5️⃣ combo × group별 rwd_cost 합계
combo_group_sum = (
    point_merged.groupby(['combo','group'])['rwd_cost']
    .sum()
    .reset_index()
)

# 6️⃣ pivot → abuse/normal 나란히 배치
combo_pivot = combo_group_sum.pivot(index='combo', columns='group', values='rwd_cost').fillna(0)
combo_pivot = combo_pivot.reset_index().rename(columns={'abuse':'sum_rwd_abuse','normal':'sum_rwd_normal'})

# 7️⃣ 어뷰징 합계 > 정상 합계 필터링 + diff 계산
combo_pivot = combo_pivot[combo_pivot['sum_rwd_abuse'] > combo_pivot['sum_rwd_normal']]
combo_pivot['diff'] = combo_pivot['sum_rwd_abuse'] - combo_pivot['sum_rwd_normal']

# 8️⃣ 차이 큰 순으로 Top10
top10_combo = combo_pivot.sort_values('diff', ascending=False).head(10)

print(top10_combo[['combo','sum_rwd_abuse','sum_rwd_normal','diff']])

plt.figure(figsize=(14,5))
x = range(len(top10_combo))
bar_width = 0.4

plt.bar([i - bar_width/2 for i in x], top10_combo['sum_rwd_abuse'], 
        width=bar_width, color='red', label='어뷰징 rwd_cost 합계')
plt.bar([i + bar_width/2 for i in x], top10_combo['sum_rwd_normal'], 
        width=bar_width, color='blue', label='정상 rwd_cost 합계')

plt.xticks(x, top10_combo['combo'], rotation=45, ha='right')
plt.title("어뷰징 합계 > 정상 합계 콤보 Top10 (광고유형-매체사 기준)")
plt.xlabel("광고유형-매체사")
plt.ylabel("rwd_cost 합계")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# -----어뷰징 디바이스 수 > 정상 디바이스 수 (참여) ---------

import pandas as pd
import matplotlib.pyplot as plt

# 1️⃣ 어뷰징 디바이스 리스트
abuse_dvc = df_cnt['dvc_idx'].unique()

# 2️⃣ part + list_1 merge
part_merged = part.merge(
    list_1[['ads_idx','ads_type']],
    on='ads_idx',
    how='left'
)

# 3️⃣ 시간대 & combo 생성
part_merged['click_date'] = pd.to_datetime(part_merged['click_date'])
part_merged['hour'] = part_merged['click_date'].dt.hour

part_merged['combo'] = (
    part_merged['ads_type'].astype(str) + "-" +
    part_merged['mda_idx'].astype(str) 
)

# 4️⃣ 그룹 라벨
part_merged['group'] = part_merged['dvc_idx'].isin(abuse_dvc).map({True:'abuse', False:'normal'})

# 5️⃣ combo × group별 고유 디바이스 수 집계
combo_device_stats = (
    part_merged.groupby(['combo','group'])['dvc_idx']
    .nunique()
    .reset_index()
)

# 6️⃣ pivot으로 abuse/normal 나란히
combo_pivot = combo_device_stats.pivot(index='combo', columns='group', values='dvc_idx').fillna(0)
combo_pivot = combo_pivot.reset_index().rename(columns={'abuse':'abuse_devices','normal':'normal_devices'})

# 7️⃣ 어뷰징 디바이스 수 > 정상 디바이스 수 필터링
combo_pivot = combo_pivot[combo_pivot['abuse_devices'] > combo_pivot['normal_devices']]

# 8️⃣ 차이(diff) 계산 & Top 10
combo_pivot['diff'] = combo_pivot['abuse_devices'] - combo_pivot['normal_devices']
top10_combo = combo_pivot.sort_values('diff', ascending=False).head(10)

print(top10_combo[['combo','abuse_devices','normal_devices','diff']])


plt.figure(figsize=(14,5))
bar_width = 0.4
x = range(len(top10_combo))

plt.bar([i - bar_width/2 for i in x], top10_combo['abuse_devices'], width=bar_width, color='red', label='어뷰징')
plt.bar([i + bar_width/2 for i in x], top10_combo['normal_devices'], width=bar_width, color='blue', label='정상')

plt.xticks(x, top10_combo['combo'], rotation=45, ha='right')
plt.title("어뷰징 디바이스 수 > 정상 디바이스 수 콤보 Top10 (참여)")
plt.xlabel("광고유형-매체사-시간대")
plt.ylabel("고유 디바이스 수")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#-------------------------------------------------------------------------------------------------------
# ----------(전체 콤보들 중에서 위 지표들로 위험도 라벨링 후) 해당 콤보별 카테고리-요일-시간대 어뷰징 리워드 총합 높은순 -------

import pandas as pd
import matplotlib.pyplot as plt

# 1️⃣ 어뷰징 디바이스 리스트
abuse_dvc = df_cnt['dvc_idx'].unique()

# 2️⃣ point + list_1 merge (ads_type, ads_category 붙이기)
point_merged = point.merge(
    list_1[['ads_idx','ads_type','ads_category']], 
    on='ads_idx', 
    how='left'
)

# 3️⃣ combo(광고유형-매체사) & 요일/시간대 생성
point_merged['regdate'] = pd.to_datetime(point_merged['regdate'])
point_merged['hour'] = point_merged['regdate'].dt.hour
point_merged['weekday'] = point_merged['regdate'].dt.day_name()

point_merged['combo'] = point_merged['ads_type'].astype(str) + "-" + point_merged['mda_idx'].astype(str)

# 4️⃣ 어뷰징 그룹만 필터
abuse_data = point_merged[point_merged['dvc_idx'].isin(abuse_dvc)]

# 5️⃣ combo = 12-270 필터
combo_12_270 = abuse_data[abuse_data['combo'] == '11-270']

# 6️⃣ 카테고리-요일-시간대별 리워드 총합 계산
cat_week_hour_sum = (
    combo_12_270.groupby(['ads_category','weekday','hour'])['rwd_cost']
    .sum()
    .reset_index()
    .sort_values('rwd_cost', ascending=False)
)

# 7️⃣ Top10 추출
top10_cat_week_hour = cat_week_hour_sum.head(10)

print(top10_cat_week_hour)

# 8️⃣ 시각화
plt.figure(figsize=(14,5))
labels = [f"{cat}-{wd}-h{h}" for cat, wd, h in zip(top10_cat_week_hour['ads_category'], top10_cat_week_hour['weekday'], top10_cat_week_hour['hour'])]
plt.bar(labels, top10_cat_week_hour['rwd_cost'], color='red')
plt.xticks(rotation=45, ha='right')
plt.title("콤보 12-270 카테고리-요일-시간대별 어뷰징 리워드 총합 Top10")
plt.xlabel("카테고리-요일-시간대")
plt.ylabel("어뷰징 rwd_cost 총합")
plt.tight_layout()
plt.show()
